In [5]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

from combine_ratios import *


In [3]:
# format: Ex    x   y
ratios = np.loadtxt("ratios.txt")

# aribrarity starting at a "later" point
ratios = ratios[20:, :]

Ex, groups = group_by(ratios)
Ngroups = len(groups)
fits = [curve_fit(fit_func, group[:, 0], group[:, 1])
        for group in groups]

xs = [group[:, 0] for group in groups]

scaling = np.ones(Ngroups)
for i, x in enumerate(xs):
    if i == Ngroups-1:
        break
    xmean = np.mean(x)
    popt_first = fits[i][0]
    popt_next = fits[i+1][0]

    yfirst = fit_func(xmean, *popt_first) * scaling[i]
    ynext = fit_func(xmean, *popt_next) * scaling[i+1]

    scaling[i+1] = yfirst/ynext

In [37]:
def plot(gsf_scaling=0.05):
    fig, ax = plt.subplots()
    
    gsf_true = np.loadtxt("../gsf.txt")
    ax.plot(gsf_true[:, 0], gsf_true[:, 1]*gsf_scaling, "k--", alpha=0.7, label="input")
    
    fit_values_x = np.array([])
    fit_values_y = np.array([])
#     for Ex_tmp, x, (popt, pcov), scale in zip(Ex, xs, fits, scaling):
#         fit_values_x = np.append(fit_values_x, x)
#         fit_values_y = np.append(fit_values_y, fit_func(x, *popt) * scale)
#         ax.plot(x, fit_func(x, *popt) * scale, alpha=0.5, label=f"{Ex_tmp}")
#         # print(np.mean(x))
        
    for Ex_tmp, group, scale in zip(Ex, groups, scaling):
        ax.plot(group[:, 0], group[:, 1] * scale, alpha=0.5, label=f"{Ex_tmp}")
        # print(np.mean(x))

    fig.legend(ncol=5)
    ax.set_yscale("log")

In [38]:
interact(plot, gsf_scaling=(0.05, 0.2, 0.01))


interactive(children=(FloatSlider(value=0.05, description='gsf_scaling', max=0.2, min=0.05, step=0.01), Output…

<function __main__.plot(gsf_scaling=0.05)>